- 联合分布的对数似然为 $\log p(x_1, x_2, P)$。其 ELBO 定义为：
    - $\mathcal{L}(q) = \mathbb{E}_{q(W, z_1, z_2)}[\log p(x_1, x_2, P, W, z_1, z_2)] - \mathbb{E}_{q(W, z_1, z_2)}[\log q(W, z_1, z_2)]$
    - $q(W, z_1, z_2) = q(W) q(z_1) q(z_2)$
    - $\mathcal{L}(q) = \mathbb{E}_{q(W)q(z_1)q(z_2)} [ \log p(x_1|z_1) + \log p(x_2|z_2) + \log p(P|z_1, z_2, W) ] - D_{KL}(q(W, z_1, z_2) || p(W, z_1, z_2))$
        - $D_{KL}(q(W, z_1, z_2) || p(W, z_1, z_2)) = D_{KL}(q(W) || p(W)) + D_{KL}(q(z_1) || p(z_1)) + D_{KL}(q(z_2) || p(z_2))$
    - $z$：分两部分一部分用来生成（重构数据$x$），一部分用来生成 preference $P$

$$
\begin{split}
\mathcal{L} &= \underbrace{\mathbb{E}_{q(z_{1,x})}[\log p(x_1|z_{1,x})] + \mathbb{E}_{q(z_{2,x})}[\log p(x_2|z_{2,x})]}_{\text{reconstruction}} \\
&+ \underbrace{\mathbb{E}_{q(W)q(z_{1,p})q(z_{2,p})}[\log p(P|z_{1,p}, z_{2,p}, W)]}_{\text{Preference}} \\
&- \underbrace{D_{KL}(q(W)||p(W)) - D_{KL}(q(z_1)||p(z_1)) - D_{KL}(q(z_2)||p(z_2))}_{\text{KL reg}}
\end{split}
$$



### modeling

- 每一个“场景”（scene）$x$ 都有一个对应的低维潜变量（latent variable）$z$，$z$ 被解耦为两部分
    - $z^{\text{recon}}$
    - $z^{\text{pref}}$
- 模型假设存在一个全局的、所有场景共享的“偏好基”（preference basis）$W$, 它定义了如何从 $z^{\text{pref}}$ 计算出偏好得分。
- 生成过程
    - 采样全局偏好基: 从一个标准正态分布先验中采样全局偏好权重矩阵 $W \in \mathbb{R}^{n_{\text{pref}} \times m}$，其中 $n_{\text{pref}}$ 是偏好潜变量的维度，$m$ 是奖励因子的数量。
        - $p(W) = \mathcal{N}(W; 0, I)$
    - 对于数据集中的每一个独立场景 $k$,
        - $p(z_k) = \mathcal{N}(z_k; 0, I)$，采样该场景的潜变量 $z_k\in \mathbb R^n$
        - 将 $z_k$ 分解为：$z_k = [z_k^{\text{pref}}, z_k^{\text{recon}}]$
        - 生成场景观测值: 通过一个解码器网络 $g_\theta(\cdot)$ 从 $z_k^{\text{recon}}$ 成场景的向量化表示 $x_k$，$p(x_k | z_k; \theta) = p(x_k | z_k^{\text{recon}}; \theta) = \mathcal{N}(x_k; g_\theta(z_k^{\text{recon}}), \sigma_x^2 I)$
    - $(x_i, x_j)$
        - 偏好结果 $y_{ij}=1$ 的概率取决于 $z_i^{\text{pref}}$ 和 $z_j^{\text{pref}}$ 的差异，并通过全局偏好基 $W$ 进行投影。
        - $W^{{n_{\text{pref}} \times m}}=V^{{n_{\text{pref}} \times m}}R^{m\times m}$
        - $s(z_i, z_j, V) = \mathbf{1}^T V^T (z_i^{\text{pref}} - z_j^{\text{pref}})$
        - 偏好概率由一个Sigmoid函数给出，这是一个伯努利似然 $p(y_{ij}=1 | z_i, z_j, W) = \sigma(s(z_i, z_j, V))$
            - binary_cross_entropy_with_logits

------

我们拥有观测数据 $\mathcal{D} = \{ (x_i, x_j) \}_{i,j}$，目标是推断出所有潜变量 $Z=\{z_k\}_k$ 和 $W$ 的后验分布 $p(Z, W | \mathcal{D})$，并学习模型参数 $\theta$。
- 模型假设 $q$ 是完全因子分解的，$q_{\phi}(Z, W) = q(W; M, \log\Sigma^2) \prod_{k=1}^{N} q(z_k; \mu_k, \log\sigma_k^2)$
    - $q(z_k) = \mathcal{N}(z_k; \mu_k, \text{diag}(\exp(\log\sigma_k^2)))$，是每个场景潜变量的近似后验，其均值 $\mu_k$ 和对数方差 $\log\sigma_k^2$ 是需要学习的参数;
        - 在代码中由 `z_latents` 这个 `nn.Embedding` 层存储）。
    - $q(W) = \mathcal{N}(W; M, \text{diag}(\exp(\log\Sigma^2)))$，是全局偏好基的近似后验，其均值 $M$ 和对数方差 $\log\Sigma^2$，也是待学习的参数
        - 代码中的 `W_mu` 和 `W_log_var`
    - $\phi = \{ \{\mu_k, \log\sigma_k^2\}_k, M, \log\Sigma^2 \}$ 是所有变分参数的集合。

目标是最大化证据下界 (Evidence Lower Bound, ELBO) $\mathcal{L}(\theta, \phi)$，它由以下几项构成：

$$
\mathcal{L}(\theta, \phi) = \mathbb{E}_{q_{\phi}}[\log p(\mathcal{D}, Z, W | \theta)] - \mathbb{E}_{q_{\phi}}[\log q_{\phi}(Z, W)]
$$

展开后，对于一个批次中的偏好对 $(x_i, x_j)$，负的ELBO（即损失函数）可以写为：

$$
\text{Loss} = -\mathcal{L} = \underbrace{\mathbb{E}_q[-\log p(x_i|z_i) - \log p(x_j|z_j)]}_{\text{Reconstruction Loss}} + \underbrace{\mathbb{E}_q[-\log p(y_{ij}=1|z_i,z_j,W)]}_{\text{Preference Loss}} + \underbrace{\text{KL}(q(z_i)||p(z_i)) + \text{KL}(q(z_j)||p(z_j))}_{\text{KL Divergence for } z} + \underbrace{\text{KL}(q(W)||p(W))}_{\text{KL Divergence for } W}
$$

- `recon_loss`, `pref_loss`, `kld_z` 和 `kld_w`。

### em-vae

- E-Step (期望步)
    - 目标: 固定模型参数 $\theta$（解码器权重），优化变分参数 $\phi$（即 `z_latents`, `W_mu`, `W_log_var`)）
        - $q(Z, W)$ 更好地逼近当前的真实后验 $p(Z, W | \mathcal{D}, \theta)$
- M-Step (最大化步)
    - 目标: 固定变分参数 $\phi$，优化模型参数 $\theta$(解码器权重)。
    - 过程: 在内循环结束后，optim_model 优化器被调用。它最小化的损失函数是 m_step_loss，该损失只包含重构损失和偏好损失。这是因为KL散度项与模型参数 $\theta$ 无关；
        - 这一步是在最大化在当前近似后验 $q$ 下，数据对数似然的期望值 $\mathbb{E}_{q_{\phi}}[\log p(\mathcal{D}, Z, W | \theta)]$